In [12]:
# import libraries we need
%pylab inline
import tensorflow as tf
from tensorflow import keras
import tensorflow_probability as tfp
tfd = tfp.distributions

import sys
sys.path.append('./../py_programs/')
from Simulator import *
from MakeModel import *
from keract import get_activations

Populating the interactive namespace from numpy and matplotlib


### load data from files

In [52]:
# load the data from files
filenames = []
x_ = []
y_ = []

# iterate over all files
#filenames = ['./../simulation/data/laser'] + ['./../simulation/data/sps']
filenames = ['./../simulation/data/halfhalf_rand/'+str(i) for i in range(50)]


for filename in filenames: 
    data = load_data(filename=filename)
    x_.append(data)
    
x_ = np.array(x_)

In [53]:
# convert data into correct shape
x_ = x_.reshape(x_.shape[0]*x_.shape[1],x_.shape[2],x_.shape[3])

# get the ground truth
threshold = 0.5
y_ = get_truth(x_, thr=threshold)

# we only need the g2signal for training
x_ = x_[:,2,:]

Now we've collected all the datasets. For training the model, we need to **shuffle** all data sets first


In [54]:
# shuffle
i = np.random.permutation(len(x_))
x_ = x_[i]
y_ = y_[i]

# reshape
x_ = x_.reshape(x_.shape[0], x_.shape[1], 1)
y_ = y_.reshape(y_.shape[0])

# allocate
x_train = x_[100:]
y_train = y_[100:]
x_test = x_[:100]
y_test = y_[:100]

In [55]:
# check the sps rate (data sets considered as sps out of all data sets) 
y_[y_==1].shape[0]/y_.shape[0]

0.585

---
# CNN model

In [73]:
# create a model
model = make_model(x_train.shape[1:])


In [57]:
epochs = 30
batch_size = 5

callbacks = set_callback(save=True,name='model_3')
model_compile(model)

<bound method Model.compile of <tensorflow.python.keras.engine.functional.Functional object at 0x7fd76c550850>>

In [58]:
model_train(model, x_train,y_train, batch_size=5, epochs=30, callbacks=callbacks)

Epoch 1/30
686/686 [==============================] - 1s 678us/step - loss: 0.1596 - sparse_categorical_accuracy: 0.4082 - binary_accuracy: 0.7478 - val_loss: 0.0351 - val_sparse_categorical_accuracy: 0.4279 - val_binary_accuracy: 0.9776
Epoch 2/30
686/686 [==============================] - 0s 556us/step - loss: 0.0325 - sparse_categorical_accuracy: 0.4156 - binary_accuracy: 0.9736 - val_loss: 0.0189 - val_sparse_categorical_accuracy: 0.4279 - val_binary_accuracy: 0.9844
Epoch 3/30
686/686 [==============================] - 0s 530us/step - loss: 0.0190 - sparse_categorical_accuracy: 0.4023 - binary_accuracy: 0.9845 - val_loss: 0.0124 - val_sparse_categorical_accuracy: 0.4279 - val_binary_accuracy: 0.9905
Epoch 4/30
686/686 [==============================] - 0s 555us/step - loss: 0.0133 - sparse_categorical_accuracy: 0.4242 - binary_accuracy: 0.9883 - val_loss: 0.0093 - val_sparse_categorical_accuracy: 0.4279 - val_binary_accuracy: 0.9932
Epoch 5/30
686/686 [============================

In [59]:
visualize_layer(model,x_train)

[('input_3', '->', (4900, 200, 1), '- Numpy array'),
 ('lambda_4', '->', (4900, 200, 1), '- Numpy array'),
 ('max_pooling1d_4', '->', (4900, 40, 1), '- Numpy array'),
 ('lambda_5', '->', (4900, 40, 1), '- Numpy array'),
 ('conv1d_2', '->', (4900, 40, 4), '- Numpy array'),
 ('batch_normalization_2', '->', (4900, 40, 4), '- Numpy array'),
 ('re_lu_2', '->', (4900, 40, 4), '- Numpy array'),
 ('max_pooling1d_5', '->', (4900, 8, 4), '- Numpy array'),
 ('flatten_2', '->', (4900, 32), '- Numpy array'),
 ('dense_4', '->', (4900, 10), '- Numpy array'),
 ('dense_5', '->', (4900, 1), '- Numpy array')]

In [60]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 200, 1)]          0         
_________________________________________________________________
lambda_4 (Lambda)            (None, 200, 1)            0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 40, 1)             0         
_________________________________________________________________
lambda_5 (Lambda)            (None, 40, 1)             0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 40, 4)             16        
_________________________________________________________________
batch_normalization_2 (Batch (None, 40, 4)             16        
_________________________________________________________________
re_lu_2 (ReLU)               (None, 40, 4)             0   

We print out the first 15 values to see how accurate the model is:

In [69]:
r=np.round(model.predict(x_test[:15]))
print(np.squeeze(r))
print(y_test[:15])


[1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0.]
[1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0.]


In [72]:
print('the test data size:',x_test.shape[0])
%timeit prediction=model.predict(x_test)

the test data size: 100
18.2 ms ± 70.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [71]:
model.evaluate(x_test,y_test)

4/4 [==============================] - 0s 994us/step - loss: 0.0074 - sparse_categorical_accuracy: 0.4000 - binary_accuracy: 1.0000


[0.007359575480222702, 0.4000000059604645, 1.0]

As we can see, to predict 100 samples this models takes less than 20 ms with a very high accuracy.